# SDXL Dreambooth Finetuning with MonsterTuner API

MonsterTuner is a no-code LLM finetuner for up to 10X more efficient and cost-effective finetuning of AI models for your business use-cases.

#### Supported Models for Finetuning:

1. LLM (Large Language Model) - For use-cases like chat completion, summary generation, sentiment analysis etc.
2. Whisper - For speech to text transcription improvement.
3. SDXL Dreambooth - Fine-tune Stable Diffusion model for customized image generation.

This Colab Notebook displays a programmatic approach for finetuning SDXL model for custom image generation and serving the finetuned model as an API using MonsterTuner.

### To get started please sign up on [Monster API](https://monsterapi.ai/signup).

#### For more info, you may explore our:
1. [Developer Docs](https://developer.monsterapi.ai/docs/launch-an-sdxl-finetuning-job) on how to launch an SDXL Finetuning Job
2. [Complete Guide](https://blog.monsterapi.ai/finetune-sdxl/) on Finetunng Stable Diffusion on your own images.

In [ ]:
!pip install monsterapi==1.0.8
from IPython.display import HTML

Copy your Auth Key from [MonsterAPI Dashboard](https://monsterapi.ai/user/dashboard) and Paste it below:


In [ ]:
import os
from google.colab import files
import requests
import json
import mimetypes
from monsterapi import client as mclient

os.environ['MONSTER_API_KEY'] = 'PROVIDE_MONSTER_API_KEY'

### Upload your Image Dataset as a ZIP File for SDXL Fine-tuning

To fine-tune the SDXL model, start by uploading your image dataset, which should be compiled into a single ZIP file (with NO internal folders).

Then we use `upload_file_to_api` function to push ZIP of images to MonsterAPI platform for using it at the time of fine-tuning.

In [ ]:
def upload_file_to_api(file_path):
    url = "https://api.monsterapi.ai/v1/upload"
    headers = {
        "accept": "application/json",
        "authorization": f"Bearer {os.environ['MONSTER_API_KEY']}"
    }

    file_name = os.path.basename(file_path)
    # Get URLs for uploading
    get_file_urls = requests.get(f"{url}?filename={file_name}", headers=headers)
    response_json = json.loads(get_file_urls.text)
    upload_url = response_json['upload_url']
    download_url = response_json['download_url']

    # Read file content as binary data
    with open(file_path, 'rb') as file_data:
        data = file_data.read()

    # Upload file to S3
    upload_headers = {
        "Content-Type": mimetypes.guess_type(file_path)[0] or 'application/octet-stream',
    }
    response = requests.put(upload_url, data=data, headers=upload_headers)

    if response.status_code == 200:
        print("File uploaded successfully!")
        return download_url
    else:
        print("Failed to upload file")
        return None

# Upload widget
uploaded = files.upload()
for file_name in uploaded.keys():
    if file_name.endswith('.zip'):
        file_path = '/content/' + file_name
        download_url = upload_file_to_api(file_path)
        if download_url:
            print(f"Download URL for {file_name}: {download_url}")
    else:
        print(f"Only ZIP files are allowed. The file '{file_name}' is not a ZIP file and will not be uploaded.")

### Launch Finetuning Job

The `launch_payload` defines configurations for the model, including details like model name, image prompt, learning parameters, and data source settings.
The model is fine-tuned using these settings, and upon completion, the session details, including a deployment ID, are printed.

In [ ]:
client = mclient(api_key=os.environ.get("MONSTER_API_KEY"))

launch_payload = {
    "dreambooth_config": {
        "model_name": "stabilityai/stable-diffusion-xl-base-1.0",
        "prompt": "Wolf in Full Moon Light",
        "learning_rate": 0.0001,
        "num_steps": 500,
        "gradient_accumulation": 4,
        "resolution": 1024,
        "scheduler": "constant"
    },
    "huggingface_config": { "push_to_hub": False },

    "dataset_config": {
    "data_source_type": "s3_presigned_link",
    "s3_presigned_url": download_url
  }
}

ret = client.finetune(service="text2image/sdxl-dreambooth", params=launch_payload)
deployment_id = ret.get("deployment_id")
print(ret)

### Fetch your Finetuning Status:

Wait until the status is `Live`. It should take 5-10 minutes. You will get an email when the Status changes. You can also check the job progress in your MonsterAPI Dashboard

In [ ]:
# Get deployment status
status_ret = client.get_deployment_status(deployment_id)
print(status_ret)

------

### Get Finetuning Job Logs

To see your finetuning job progress, please run the cell below

In [ ]:
# Get deployment logs
logs_ret = client.get_deployment_logs(deployment_id)
print(logs_ret)

------

### Terminate Finetuning Job

CAUTION: If you wish to terminate your finetuning job, please uncomment and run the cell below

In [ ]:
## Terminate Deployment
# terminate_return = client.terminate_deployment(deployment_id)
# print(terminate_return)

### Deploy your Finetuned SDXL Model

You can deploy your Finetuned Model as a sharable Gradio UI in a single command once the Finetuning job is Completed

In [ ]:
status_ret = client.get_deployment_status(deployment_id)
loramodel_path = status_ret['info']['model_url']
print("Model URL:", loramodel_path)

deployment_payload = {
    "deployment_name": "test_client_sdxl_dreambooth_deployment",
    "basemodel_path": "stabilityai/stable-diffusion-xl-base-1.0",
    "loramodel_path": loramodel_path
    }

# Launch a deployment
ret = client.deploy("sdxl-dreambooth", deployment_payload)
inference_deployment_id = ret.get("deployment_id")
print(ret)

### Fetch your Deployment Status:

Wait until the status is `Live`. It should take 5-10 minutes.

In [ ]:
# Get deployment status
status_ret = client.get_deployment_status(inference_deployment_id)
print(status_ret)

### Play with your Finetuned Model

---
Once you have finetuned SDXL model, you can test it out and integrate it into your application by deploying it as an endpoint. To generate images with your finetuned image generation model, let us deploy the model with Gradio UI:


In [ ]:
Gradio_URL = status_ret['URL']
print("\nMonsterAPI Gradio UI: ",Gradio_URL)
HTML(f'<iframe src="{Gradio_URL}" width="100%" height="500px"></iframe>')

------

### Get Deployment Job Logs

To see your deployment job logs, please run the cell below

In [ ]:
# Get deployment logs
logs_ret = client.get_deployment_logs(inference_deployment_id)
print(logs_ret)

------

### Terminate Deployment

CAUTION: If you wish to terminate your live deployment, please uncomment and run the cell below

In [ ]:
## Terminate Deployment
# terminate_return = client.terminate_deployment(deployment_id)
# print(terminate_return)